In [1]:
import pandas as pd
import datetime
from konlpy.tag import Okt

In [2]:
data = pd.read_csv("../data/pohang_token.csv")

/tmp/ipykernel_5193/145161895.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/pohang_token.csv")


In [7]:
token_lst = data['tokens']

In [28]:
import ast

token_lst = data['tokens'].apply(lambda x : ast.literal_eval(x))

In [37]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=token_lst, window=30, min_count=10, workers=8, sg=0)

In [27]:
from gensim.models import KeyedVectors
model.save(f'..//daum_{datetime.datetime.now().strftime("%m_%d_%H")}_w2v_30_10') # 모델 저장

In [38]:
model.wv.most_similar("포항", topn = 30)

[('구미', 0.5341535806655884),
 ('울진', 0.5147035121917725),
 ('대구', 0.4891129732131958),
 ('대한항공', 0.4706384539604187),
 ('청도', 0.46877026557922363),
 ('서울', 0.4639595150947571),
 ('순천', 0.4610622227191925),
 ('경주', 0.4610317349433899),
 ('영천', 0.45766884088516235),
 ('인천', 0.4557476341724396),
 ('안동', 0.44364944100379944),
 ('경산', 0.4412010908126831),
 ('외국인', 0.44033387303352356),
 ('광양', 0.43733736872673035),
 ('울산', 0.4326839745044708),
 ('출국', 0.4320051670074463),
 ('비자', 0.43037161231040955),
 ('본사', 0.42927464842796326),
 ('고령', 0.4289134442806244),
 ('경기', 0.4282273054122925),
 ('김포', 0.426418662071228),
 ('강릉', 0.42351678013801575),
 ('마라톤', 0.4230053722858429),
 ('대전', 0.4196406900882721),
 ('김태훈', 0.4157894253730774),
 ('시외', 0.4143136143684387),
 ('군산', 0.41359713673591614),
 ('여수', 0.41344237327575684),
 ('청라', 0.41058632731437683),
 ('대회', 0.40964242815971375)]

# APriori

In [39]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()
te_result = te.fit(token_lst).transform(token_lst)

In [42]:
key_df = pd.DataFrame(te_result, columns=te.columns_)

itemset = apriori(key_df, min_support=0.3, use_colnames=True)

from mlxtend.frequent_patterns import association_rules
apriori_result = association_rules(itemset, metric="confidence", min_threshold=0.5)

/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/mlxtend/frequent_patterns/association_rules.py:112: RuntimeWarning: invalid value encountered in true_divide
  "confidence": lambda sAC, sA, _: sAC / sA,
/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/mlxtend/frequent_patterns/association_rules.py:112: RuntimeWarning: invalid value encountered in true_divide
  "confidence": lambda sAC, sA, _: sAC / sA,
/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/mlxtend/frequent_patterns/association_rules.py:92: RuntimeWarning: invalid value encountered in true_divide
  confidence = sAC / sA


In [64]:
temp = apriori_result[apriori_result['antecedents'] == frozenset({'포항시'})].sort_values('confidence', ascending = False)

In [69]:
temp.columns

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction'],
      dtype='object')

In [72]:
temp[['antecedents', 'consequents', 'confidence']].head(300).to_csv("../pohang_apriori.csv")